In [24]:
%pip install pandas_datareader pykrx yfinance

# src

## common

### path.py

In [25]:
class PATH:
    import os

    try:
        ROOT = os.path.dirname(__file__)
        while not ROOT.endswith('yuho'):
            ROOT = os.path.dirname(ROOT)
    except NameError:
        ROOT = 'https://raw.githubusercontent.com/labwons/yuho/main/'

    try:
        DESKTOP = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
        DOWNLOADS = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Downloads')
    except KeyError:
        DESKTOP = DOWNLOADS = ROOT

    GROUP  = os.path.join(ROOT, r'src/fetch/market/json/group.json')
    STATE  = os.path.join(ROOT, r'src/fetch/market/json/state.json')
    SPECS  = os.path.join(ROOT, r'src/fetch/market/json/specs.json')
    INDEX  = os.path.join(ROOT, r'src/fetch/market/json/index.json')

    BUBBLE = os.path.join(ROOT, r'docs/src/json/bubble.json')
    MAP    = os.path.join(ROOT, r'docs/src/json/treemap.json')
    MACRO  = os.path.join(ROOT, r'docs/src/json/macro.json')



if __name__ == "__main__":
    print(PATH.ROOT)
    print(PATH.GROUP)
    print(PATH.STATE)
    print(PATH.SPECS)
    print(PATH.INDEX)
    print(PATH.MAP)

https://raw.githubusercontent.com/labwons/yuho/main/
https://raw.githubusercontent.com/labwons/yuho/main/src/fetch/market/json/group.json
https://raw.githubusercontent.com/labwons/yuho/main/src/fetch/market/json/state.json
https://raw.githubusercontent.com/labwons/yuho/main/src/fetch/market/json/specs.json
https://raw.githubusercontent.com/labwons/yuho/main/src/fetch/market/json/index.json
https://raw.githubusercontent.com/labwons/yuho/main/docs/src/json/treemap.json


### syscon.py

In [32]:
from typing import Dict

class WISE:
    SECTOR_CODE:Dict[str, str] = {
        'WI100': '에너지', 'WI110': '화학',
        'WI200': '비철금속', 'WI210': '철강', 'WI220': '건설', 'WI230': '기계', 'WI240': '조선', 'WI250': '상사,자본재', 'WI260': '운송',
        'WI300': '자동차', 'WI310': '화장품,의류', 'WI320': '호텔,레저', 'WI330': '미디어,교육', 'WI340': '소매(유통)',
        'WI400': '필수소비재', 'WI410': '건강관리',
        'WI500': '은행', 'WI510': '증권', 'WI520': '보험',
        'WI600': '소프트웨어', 'WI610': 'IT하드웨어', 'WI620': '반도체', 'WI630': 'IT가전', 'WI640': '디스플레이',
        'WI700': '통신서비스',
        'WI800': '유틸리티'
    }
    INDEX_CODE = SECTOR_CODE.copy()
    INDEX_CODE.update({'1001': 'KOSPI', '2001': 'KOSDAQ'})
    CODE_LABEL:Dict[str, str] = {
        'CMP_CD': 'ticker', 'CMP_KOR': 'name',
        'SEC_CD': 'sectorCode', 'SEC_NM_KOR': 'sectorName',
        'IDX_CD': 'industryCode', 'IDX_NM_KOR': 'industryName',
    }


class KRX:
    CAP_LABEL:Dict[str, str] = {
        '종가':'close', '시가총액':'marketCap',
        '거래량':'volume', '거래대금':'amount', '상장주식수':'shares'
    }
    MUL_LABEL:Dict[str, str] = {
        'PER': 'PER(Q)', 'PBR': 'PBR(Q)', 'DIV': 'DIV(Q)'
    }
    PCT_LABEL:Dict[str, str] = {"지분율":'foreignRate'}
    PRC_LABEL:Dict[str, str] = {
        "시가":"open", "고가":"high", "저가":"low", "종가":"close",
        "거래량":"volume", "거래대금":"amount"
    }

## fetch

### market

#### group.py

In [27]:
# import PATH, WISE
from pandas import (
    DataFrame,
    concat,
    Index,
    read_json
)
from pykrx.stock import get_index_portfolio_deposit_file
from re import compile
from requests import get
from requests.exceptions import JSONDecodeError
from time import sleep
from typing import (
    Iterable,
    List
)

class MarketGroup(DataFrame):

    _log:List[str] = []

    def __init__(self, update:bool=True):
        if not update:
            super().__init__(read_json(PATH.GROUP, orient='index'))
            self.index = self.index.astype(str).str.zfill(6)
            self.index.name = 'ticker'
            return

        date = self.fetchTradingDate()
        self.log = f'Begin [Market Group Fetch] @{date}'
        objs = []
        for n, (code, name) in enumerate(WISE.SECTOR_CODE.items()):
            obj = self.fetchWiseGroup(code, date)
            objs.append(obj)
            proc = f"... {n + 1} / {len(WISE.SECTOR_CODE)} : {code} {name} :: "
            self.log = f"{proc}Fail" if obj.empty else f"{proc}Success"

        super().__init__(concat(objs, axis=0, ignore_index=True))
        self.drop(columns=[key for key in self if not key in WISE.CODE_LABEL], inplace=True)
        self.drop(index=self[self['SEC_CD'].isna()].index, inplace=True)
        self.rename(columns=WISE.CODE_LABEL, inplace=True)
        self.set_index(keys="ticker", inplace=True)
        self['industryName'] = self['industryName'].str.replace("WI26 ", "")

        kq, lg = self.fetchKosdaqList(self.index), self.fetchLargeCapList(self.index)
        self.loc[kq, 'name'] = self.loc[kq, 'name'] + '*'
        self.loc[lg, 'stockSize'] = 'large'
        self.log = "... Identify Tickers: Success (N F/S)"

        sc_mdi = self[(self['industryCode'] == 'WI330') & (self['sectorCode'] == 'G50')].index
        sc_edu = self[(self['industryCode'] == 'WI330') & (self['sectorCode'] == 'G25')].index
        sc_sw = self[(self['industryCode'] == 'WI600') & (self['sectorCode'] == 'G50')].index
        sc_it = self[(self['industryCode'] == 'WI600') & (self['sectorCode'] == 'G45')].index
        self.loc[sc_mdi, 'industryCode'], self.loc[sc_mdi, 'industryName'] = 'WI331', '미디어'
        self.loc[sc_edu, 'industryCode'], self.loc[sc_edu, 'industryName'] = 'WI332', '교육'
        self.loc[sc_sw, 'industryCode'], self.loc[sc_sw, 'industryName'] = 'WI601', '소프트웨어'
        self.loc[sc_it, 'industryCode'], self.loc[sc_it, 'industryName'] = 'WI602', 'IT서비스'
        self.log = f'End [Market Group Fetch]'
        return

    @property
    def log(self) -> str:
        return "\n".join(self._log)

    @log.setter
    def log(self, log:str):
        self._log.append(log)

    @classmethod
    def fetchTradingDate(cls) -> str:
        URL = 'https://www.wiseindex.com/Index/Index#/G1010.0.Components'
        pattern = compile(r"var\s+dt\s*=\s*'(\d{8})'")
        return pattern.search(get(URL).text).group(1)

    @classmethod
    def fetchWiseGroup(cls, code:str, date:str="", countdown:int=5) -> DataFrame:
        resp = get(f'http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={date}&sec_cd={code}')
        try:
            return DataFrame(resp.json()['list'])
        except JSONDecodeError:
            if countdown == 0:
                return DataFrame()
            sleep(5)
            return cls.fetchWiseGroup(code, date, countdown - 1)

    @classmethod
    def fetchKosdaqList(cls, _tickers:Index=None) -> List[str]:
        tickers = get_index_portfolio_deposit_file('2001')
        if not _tickers.empty:
            tickers = [ticker for ticker in _tickers if ticker in tickers]
        return tickers

    @classmethod
    def fetchLargeCapList(cls, _tickers:Index=None) -> List[str]:
        tickers = get_index_portfolio_deposit_file('2203') \
                + get_index_portfolio_deposit_file('1028')
        if not _tickers.empty:
            tickers = [ticker for ticker in _tickers if ticker in tickers]
        return tickers


if __name__ == "__main__":
    marketGroup = MarketGroup(True)
    print(marketGroup)
    # print(marketGroup.log)


       industryCode industryName      name sectorCode sectorName stockSize
ticker                                                                    
096770        WI100          에너지   SK이노베이션        G10        에너지     large
034730        WI100          에너지        SK        G10        에너지     large
267250        WI100          에너지      HD현대        G10        에너지     large
010950        WI100          에너지     S-Oil        G10        에너지     large
009830        WI100          에너지     한화솔루션        G10        에너지     large
...             ...          ...       ...        ...        ...       ...
053050        WI800         유틸리티     지에스이*        G55       유틸리티       NaN
003480        WI800         유틸리티  한진중공업홀딩스        G55       유틸리티       NaN
016710        WI800         유틸리티     대성홀딩스        G55       유틸리티       NaN
034590        WI800         유틸리티    인천도시가스        G55       유틸리티       NaN
017390        WI800         유틸리티      서울가스        G55       유틸리티       NaN

[2401 rows x 6 columns]


#### index.py

In [28]:
# import PATH, WISE
from pandas import (
    DataFrame,
    read_json,
    to_datetime
)
from pykrx.stock import (
    get_index_ohlcv_by_date,
    get_nearest_business_day_in_a_week
)
from re import compile, search
from requests import get
from time import sleep
from typing import (
    List
)
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)


class MarketIndex(DataFrame):

    _log:List[str] = []

    def __init__(self, update:bool=True):
        super().__init__(read_json(PATH.INDEX, orient='index'))
        self.index = self.index.date
        if not update:
            return

        trading_date = get_nearest_business_day_in_a_week()
        server_date = self.fetchServerDate()
        self.log = f'Begin [Market Index Fetch] @{trading_date}'

        for n, (code, name) in enumerate(WISE.INDEX_CODE.items()):
            proc = f'... ({n + 1} / {len(WISE.INDEX_CODE)}) : {code} {name} :: '
            start = self[code].dropna().index[-1]
            end = trading_date if code in ['1001', '2001'] else server_date
            if start == end:
                continue
            fetch = self.fetchWiseSeries(code, f'{start}', f'{end}')
            if fetch.empty:
                self.log = f'{proc}Fail'
                continue
            for dt in fetch.index:
                self.at[dt, code] = fetch.loc[dt, code]
            self.log = f'{proc}Success '
        self.log = 'End [Market Index Fetch]'
        return

    @property
    def log(self) -> str:
        return "\n".join(self._log)

    @log.setter
    def log(self, log:str):
        self._log.append(log)

    @classmethod
    def _netDate2normDate(cls, timestamp:str):
        timestamp = int(search(r'\((\d+)\)', timestamp).group(1))
        return to_datetime(timestamp, unit='ms', utc=True) \
               .tz_convert('Asia/Seoul') \
               .date()

    @classmethod
    def fetchServerDate(cls) -> str:
        URL = 'https://www.wiseindex.com/Index/Index#/G1010.0.Components'
        pattern = compile(r"var\s+dt\s*=\s*'(\d{8})'")
        return pattern.search(get(URL).text).group(1)

    @classmethod
    def fetchWiseSeries(cls, code:str, start:str, end:str, countdown:int=5) -> DataFrame:
        if code in ['1001', '2001']:
            fetch = get_index_ohlcv_by_date(start, end, code, 'd', False)
            fetch.index = fetch.index.date
            fetch = fetch.rename(columns={"종가": code})
            return fetch

        resp = get(f'http://www.wiseindex.com/DataCenter/GridData?currentPage=1&endDT={end}&fromDT={start}&index_ids={code}&isEnd=1&itemType=1&perPage=10000&term=1')
        try:
            fetch = DataFrame(resp.json())[["TRD_DT", "IDX1_VAL1"]]
            fetch["TRD_DT"] = fetch["TRD_DT"].apply(cls._netDate2normDate)
            return fetch.rename(columns={"IDX1_VAL1": code}).set_index(keys="TRD_DT")
        except JSONDecodeError:
            if countdown == 0:
                return DataFrame()
            sleep(5)
            return cls.fetchWiseSeries(code, start, end, countdown - 1)


if __name__ == "__main__":
    marketIndex = MarketIndex(True)
    print(marketIndex)
    print(marketIndex.log)


               1001     2001    WI100    WI110    WI200    WI210    WI220  \
2000-01-04  1059.04  2660.00  1000.00  1000.00  1000.00  1000.00  1000.00   
2000-01-05   986.31  2629.50   950.14   965.05  1039.95  1046.55   991.11   
2000-01-06   960.79  2475.20   908.20   920.33  1003.60  1009.53   939.19   
2000-01-07   948.65  2276.60   890.47   963.16   999.57  1015.77   939.90   
2000-01-10   987.24  2310.80   920.38   990.70  1013.04  1004.84   939.74   
...             ...      ...      ...      ...      ...      ...      ...   
2025-02-04  2481.69   719.92  3361.25  3425.13  4716.14  1650.64   991.98   
2025-02-03  2453.95   703.80  3330.46  3444.99  4670.12  1701.24   981.65   
2025-01-31  2517.37   728.29  3457.60  3641.31  4852.16  1778.57   998.98   
2025-01-24  2536.80   728.74  3471.90  3661.58  4955.04  1790.32  1011.70   
2025-02-05  2509.27   730.98      NaN      NaN      NaN      NaN      NaN   

              WI230    WI240    WI250  ...    WI500    WI510    WI520  \
20

#### state.py

In [23]:
# import PATH, KRX
from datetime import datetime, timedelta
from pandas import (
    concat,
    DataFrame,
    Index,
    read_json
)
from pykrx.stock import (
    get_exhaustion_rates_of_foreign_investment,
    get_nearest_business_day_in_a_week,
    get_market_cap_by_ticker,
    get_market_fundamental,
)
from requests.exceptions import JSONDecodeError, SSLError


class MarketState(DataFrame):

    _log:List[str] = []

    def __init__(self, update:bool=True):
        if not update:
            super().__init__(read_json(PATH.STATE, orient='index'))
            return

        objs = []
        date = get_nearest_business_day_in_a_week()
        self.log = f'Begin [Market State Fetch] @{date}'

        marketCap = self.fetchMarketCap(date)
        objs.append(marketCap)
        self.log = f'... Fetch Market Cap :: {"Fail" if marketCap.empty else "Success"}'

        multiples = self.fetchMultiples(date)
        objs.append(multiples)
        self.log = f'... Fetch Multiples :: {"Fail" if multiples.empty else "Success"}'

        foreignRate = self.fetchForeignRate(date)
        objs.append(foreignRate)
        self.log = f'... Fetch Foreign Rate :: {"Fail" if foreignRate.empty else "Success"}'

        super().__init__(concat(objs, axis=1))
        self.drop(index=self.fetchKonexList(date), inplace=True)

        self.log = 'End [Market State Fetch]'
        return

    @property
    def log(self) -> str:
        return "\n".join(self._log)

    @log.setter
    def log(self, log:str):
        self._log.append(log)

    @classmethod
    def fetchMarketCap(cls, date:str) -> DataFrame:
        try:
            df = get_market_cap_by_ticker(date=date,market='ALL') \
                 .rename(columns=KRX.CAP_LABEL)
            df.index.name = 'ticker'
            return df
        except (KeyError, RecursionError, JSONDecodeError, SSLError):
            return DataFrame(columns=list(KRX.CAP_LABEL.values()))

    @classmethod
    def fetchKonexList(cls, date:str) -> Index:
        try:
            return get_market_cap_by_ticker(date=date, market='KONEX').index
        except (KeyError, RecursionError, JSONDecodeError, SSLError):
            return []

    @classmethod
    def fetchMultiples(cls, date:str) -> DataFrame:
        try:
            df = get_market_fundamental(date=date, market="ALL") \
                 .rename(columns=KRX.MUL_LABEL)
            df.index.name = "ticker"
            return df[KRX.MUL_LABEL.values()]
        except (KeyError, RecursionError, JSONDecodeError, SSLError):
            return DataFrame(columns=KRX.MUL_LABEL.values())

    @classmethod
    def fetchForeignRate(cls, date:str) -> DataFrame:
        try:
            df = get_exhaustion_rates_of_foreign_investment(date=date, market='ALL') \
                 .rename(columns=KRX.PCT_LABEL)
            df.index.name = 'ticker'
            return round(df[KRX.PCT_LABEL.values()].astype(float), 2)
        except (KeyError, RecursionError, JSONDecodeError, SSLError):
            return DataFrame(columns=KRX.PCT_LABEL.values())

@classmethod
def fetchReturns(cls, date:str, tickers:Iterable=None) -> DataFrame:
    fdef = lambda x: get_market_cap_by_ticker(
        get_nearest_business_day_in_a_week(
            (
                datetime.strptime(date, "%Y%m%d") - timedelta(x)
            ).strftime("%Y%m%d")
        )
    ).rename(columns=KRX.CAP_LABEL)

    objs = {
        'D+0': fdef(0), 'D-1': fdef(1),
        'W-1': fdef(7),
        'M-1': fdef(30), 'M-3': fdef(91), 'M-6': fdef(182),
        'Y-1': fdef(365),
    }
    base = concat(objs=objs, axis=1)

    objs = {
        k: 100 * (base[k]['close'] / base['D+0']['close'] - 1)
        for k in objs if k != 'D+0'
    }
    returns = concat(objs=objs, axis=1)

    miss = base[base['Y-1']['shares'] != base['D+0']['shares']]

    # objs = {
    #     dt: stock.get_market_cap_by_ticker(
    #     date=str(Calendar[dt]),
    #     market='ALL',
    # )['상장주식수'] for dt in ['D-0', 'Y-1']

    # shares = concat({dt: stock.get_market_cap_by_ticker(
    #     date=str(Calendar[dt]),
    #     market='ALL',
    # )['상장주식수'] for dt in ['D-0', 'Y-1']}, axis=1)
    _shares = _shares[~_shares['D-0'].isna()]

    def _update_return(_tickers:Iterable) -> DataFrame:
        fromdate, todate = Calendar['Y-2'].strftime("%Y%m%d"), str(Calendar)
        objs = []
        for ticker in _tickers:
            src = stock.get_market_ohlcv_by_date(ticker=ticker, fromdate=fromdate, todate=todate)['종가']
            obj = {"ticker": ticker}
            for interval, date in Calendar:
                src_copy = src[src.index.date >= date]
                obj[interval] = round(100 * (src_copy.iloc[-1] / src_copy.iloc[0] - 1), 2)
            objs.append(obj)
        return DataFrame(objs).set_index(keys='ticker')

    _shares = pd.concat({dt: stock.get_market_cap_by_ticker(
        date=str(Calendar[dt]),
        market='ALL',
    )['상장주식수'] for dt in ['D-0', 'Y-1']}, axis=1)
    _shares = _shares[~_shares['D-0'].isna()]
    if not tickers is None:
        _shares = _shares[_shares.index.isin(tickers)]

    _normal = _shares[_shares['D-0'] == _shares['Y-1']].index
    _change = _shares[_shares['D-0'] != _shares['Y-1']].index
    _return = _base_return()
    return pd.concat([_return[_return.index.isin(_normal)], _update_return(_change)])


# if __name__ == "__main__":
#     marketState = MarketState(True)
#     print(marketState)





{'D-1': '20250204',
 'W-1': '20250124',
 'M-1': '20250106',
 'M-3': '20241106',
 'M-6': '20240807',
 'Y-1': '20240206'}

In [30]:
from pykrx.stock import (
    get_exhaustion_rates_of_foreign_investment,
    get_nearest_business_day_in_a_week,
    get_market_cap_by_ticker,
    get_market_fundamental,
    get_market_ohlcv_by_ticker
)
get_market_ohlcv_by_ticker('20250205')

,시가,고가,저가,종가,거래량,거래대금,등락률
티커,,,,,,,
095570,3900,4100,3900,4050,106031,426026585,5.06
006840,9980,10030,9890,9900,9608,95627980,-0.90
027410,3390,3435,3350,3350,68938,233220520,-0.59
282330,103900,105100,102700,103200,18777,1961844900,0.49
138930,12030,12100,11850,11920,705566,8448780720,0.17
...,...,...,...,...,...,...,...
079980,2510,2575,2510,2535,28199,71522645,0.40
005010,4340,4365,4265,4355,266913,1150616470,1.28
000540,3195,3280,3195,3230,24367,78965665,0.47
